<a href="https://colab.research.google.com/github/chiamaka249/IgboNER/blob/main/Copy_of_igbobert4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -c https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
!wget -c https://raw.githubusercontent.com/chiamaka249/lacuna_pos_ner/main/language_corpus/ibo/ibo.txt

--2021-12-06 20:13:45--  https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IgnatiusEzeani/IGBONLP/master/ig_monoling/text.zip [following]
--2021-12-06 20:13:45--  https://raw.githubusercontent.com/IgnatiusEzeani/IGBONLP/master/ig_monoling/text.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7604282 (7.3M) [application/zip]
Saving to: ‘text.zip’

text.zip            100%[===================>]   7.25M  --.-KB/s    in 0.08s   

2021-12-06 20:13:45 (85.7 MB/s) - ‘text.zip’ saved [7604282/7604282]

--2021-12-06 20:13:45--  ht

In [2]:
import zipfile
import os


def unzip(zipfilename):
  try:
    with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
      zip_ref.extractall(zipfilename[:-4])
      return f"'{zipfilename}' unzipped!"
  except FileNotFoundError:
    print(f"Cannot find '{zipfilename}' file")

unzip("text.zip")
!rm text.zip

In [3]:
#copies the file "ibo.txt" to into the folder "text"
import shutil
newPath = shutil.copy('/content/ibo.txt', '/content/text')

In [4]:
# import os
#import shutil
dir_name = "/content/text"
text=""
for fname in os.listdir(dir_name):
  fname = os.path.join(dir_name, fname)
  with open(fname, "r", encoding="utf8") as datafile:
    text = text+"\n"+datafile.read()

with open("data.txt", "w", encoding="utf8") as datafile:
  datafile.write(text)

shutil.rmtree("text")

In [5]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-x56890xj
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-x56890xj
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 71.0 MB/s 
     |████████████████████████████████| 61 kB 584 kB/s 
     |████████████████████████████████| 895 kB 92.4 MB/s 
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3255337 sha256=0784f7be12e958804c5e766d5d59667c8eb0d2594165786d43c262123ca70b26
  Stored in directory: /tmp/pip-ephem-wheel-cache-iv7d_qiz/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attemp

In [6]:
%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 20.3 s, sys: 1.23 s, total: 21.5 s
Wall time: 5.94 s


In [7]:
!mkdir igbo_bert4
tokenizer.save_model("igbo_bert4")

['igbo_bert4/vocab.json', 'igbo_bert4/merges.txt']

In [8]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./igbo_bert4/vocab.json",
    "./igbo_bert4/merges.txt",
)

In [9]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [10]:
tokenizer.encode("Simone gara ụka ụnyahụ guọ egwu ma ga-kwa taa.").tokens

['<s>',
 'Simone',
 'Ġgara',
 'Ġá»¥ka',
 'Ġá»¥nyahá»¥',
 'Ġgu',
 'á»į',
 'Ġegwu',
 'Ġma',
 'Ġga',
 '-',
 'kwa',
 'Ġtaa',
 '.',
 '</s>']

In [11]:
# Check that we have a GPU
!nvidia-smi

Mon Dec  6 20:17:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [13]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
#from google.colab import files
#files.upload()

In [14]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./igbo_bert4", max_len=512)

file ./igbo_bert4/config.json not found
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.
file ./igbo_bert4/config.json not found
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


In [15]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [16]:
model.num_parameters()
# => 83 million parameters

83504416

In [17]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "/content/data.txt",
    block_size = 128
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 33.1 s, sys: 1.27 s, total: 34.4 s
Wall time: 15.7 s


In [18]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [19]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./igbo_bert4",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [20]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 391448
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 30585
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,6.396800
1000,5.333900
1500,4.898600
2000,4.537700
2500,4.304700
3000,4.141900
3500,3.988200
4000,3.917300
4500,3.790800
5000,3.677400


Saving model checkpoint to ./igbo_bert4/checkpoint-10000
Configuration saved in ./igbo_bert4/checkpoint-10000/config.json
Model weights saved in ./igbo_bert4/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./igbo_bert4/checkpoint-20000
Configuration saved in ./igbo_bert4/checkpoint-20000/config.json
Model weights saved in ./igbo_bert4/checkpoint-20000/pytorch_model.bin
Saving model checkpoint to ./igbo_bert4/checkpoint-30000
Configuration saved in ./igbo_bert4/checkpoint-30000/config.json
Model weights saved in ./igbo_bert4/checkpoint-30000/pytorch_model.bin
Deleting older checkpoint [igbo_bert4/checkpoint-10000] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 3h 5min 26s, sys: 37min 31s, total: 3h 42min 57s
Wall time: 3h 41min 51s


TrainOutput(global_step=30585, training_loss=3.1539120062827597, metrics={'train_runtime': 13311.4889, 'train_samples_per_second': 147.034, 'train_steps_per_second': 2.298, 'total_flos': 3.356550366134784e+16, 'train_loss': 3.1539120062827597, 'epoch': 5.0})

In [21]:
trainer.save_model("./igbo_bert4")

Saving model checkpoint to ./igbo_bert4
Configuration saved in ./igbo_bert4/config.json
Model weights saved in ./igbo_bert4/pytorch_model.bin


In [22]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./igbo_bert4",
    tokenizer="./igbo_bert4"
)

loading configuration file ./igbo_bert4/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert4",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file ./igbo_bert4/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert4",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dro

In [23]:
# The sun <mask>.
# =>

fill_mask("Abụ m Maazị <mask>.") #= okafor/Ọkafọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.006366073153913021,
  'sequence': 'Abụ m Maazị A.',
  'token': 347,
  'token_str': ' A'},
 {'score': 0.0061102816835045815,
  'sequence': 'Abụ m Maazị M.',
  'token': 395,
  'token_str': ' M'},
 {'score': 0.005945019423961639,
  'sequence': 'Abụ m Maazị O.',
  'token': 377,
  'token_str': ' O'},
 {'score': 0.004409363958984613,
  'sequence': 'Abụ m Maazị Obi.',
  'token': 926,
  'token_str': ' Obi'},
 {'score': 0.003987359814345837,
  'sequence': 'Abụ m Maazị Chineke.',
  'token': 403,
  'token_str': ' Chineke'}]

In [24]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na <mask> ji na akara.") #= eri
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.12052495032548904,
  'sequence': 'Nwaanyị na ya ji na akara.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.07421518117189407,
  'sequence': 'Nwaanyị na nwunye ji na akara.',
  'token': 732,
  'token_str': ' nwunye'},
 {'score': 0.04338353872299194,
  'sequence': 'Nwaanyị na nwaanyị ji na akara.',
  'token': 620,
  'token_str': ' nwaanyị'},
 {'score': 0.03189236298203468,
  'sequence': 'Nwaanyị na- ji na akara.',
  'token': 17,
  'token_str': '-'},
 {'score': 0.0238909013569355,
  'sequence': 'Nwaanyị na nne ji na akara.',
  'token': 902,
  'token_str': ' nne'}]

In [25]:
# The sun <mask>.
# =>

fill_mask("Chineke ga- ebibikwa ndị niile na- eme ihe <mask>.") #=ọjọọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.35245102643966675,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọjọọ.',
  'token': 718,
  'token_str': ' ọjọọ'},
 {'score': 0.17003658413887024,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọma.',
  'token': 498,
  'token_str': ' ọma'},
 {'score': 0.08485106378793716,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe niile.',
  'token': 430,
  'token_str': ' niile'},
 {'score': 0.047518376260995865,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe a.',
  'token': 266,
  'token_str': ' a'},
 {'score': 0.01763184554874897,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ùgwù.',
  'token': 1300,
  'token_str': ' ùgwù'}]

In [26]:
fill_mask("O riri <mask>.")

# This is the beginning of a beautiful <mask>.
# =>

[{'score': 0.3398936986923218,
  'sequence': 'O riri ya.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.0448780357837677,
  'sequence': 'O riri mmiri.',
  'token': 703,
  'token_str': ' mmiri'},
 {'score': 0.03440803661942482,
  'sequence': 'O riri ha.',
  'token': 297,
  'token_str': ' ha'},
 {'score': 0.03390880301594734,
  'sequence': 'O riri ihe.',
  'token': 302,
  'token_str': ' ihe'},
 {'score': 0.012751125730574131,
  'sequence': 'O riri afọ.',
  'token': 485,
  'token_str': ' afọ'}]

In [27]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na eri <mask> na akara.") #= ji
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.07749678939580917,
  'sequence': 'Nwaanyị na eri ya na akara.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.026143409311771393,
  'sequence': 'Nwaanyị na eri ihe na akara.',
  'token': 302,
  'token_str': ' ihe'},
 {'score': 0.026121513918042183,
  'sequence': 'Nwaanyị na eri nri na akara.',
  'token': 873,
  'token_str': ' nri'},
 {'score': 0.014630773104727268,
  'sequence': 'Nwaanyị na eri di na akara.',
  'token': 686,
  'token_str': ' di'},
 {'score': 0.013305693864822388,
  'sequence': 'Nwaanyị na eri mmiri na akara.',
  'token': 703,
  'token_str': ' mmiri'}]

In [28]:
# The sun <mask>.
# =>

fill_mask("Ọ bụ <mask>a ka a na- arịa .") #= mmadụ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.03819141909480095,
  'sequence': 'Ọ bụ-a ka a na- arịa.',
  'token': 17,
  'token_str': '-'},
 {'score': 0.02734088897705078,
  'sequence': 'Ọ bụ Nwannaa ka a na- arịa.',
  'token': 1516,
  'token_str': ' Nwanna'},
 {'score': 0.02290385030210018,
  'sequence': 'Ọ bụ nwunyea ka a na- arịa.',
  'token': 732,
  'token_str': ' nwunye'},
 {'score': 0.01657039113342762,
  'sequence': 'Ọ bụ nwaa ka a na- arịa.',
  'token': 420,
  'token_str': ' nwa'},
 {'score': 0.011486751958727837,
  'sequence': 'Ọ bụ naanịa ka a na- arịa.',
  'token': 790,
  'token_str': ' naanị'}]

In [29]:
import shutil

In [30]:
shutil.make_archive("/content/igbo_bert4", 'zip', "igbo_bert4")

'/content/igbo_bert4.zip'

In [ ]:
from google.colab import files
files.download("/content/igbo_bert4.zip")

FileNotFoundError: ignored